In [1]:
!pip install transformers jiwer pandas accelerate -U
!pip install datasets tokenizers seqeval -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 36.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 55.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 KB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 132.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 KB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 1.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 KB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 KB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 137.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 KB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 KB 15.2 MB/s eta 0:00:00
    

In [1]:
import os
import re
from collections import Counter

import pandas as pd
import jiwer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
!pip install ipywidgets
from tqdm import tqdm

# from .autonotebook import tqdm as notebook_tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
print(device)

cuda:0


In [3]:
import os
import pandas as pd

def ingest_idrisi_data(data_dir):
    sentences, labels = [], []

    # Traverse each folder inside Train or Test
    for root, dirs, files in os.walk(data_dir):
       
        for file_name in files:  # For each file inside the current directory
           
            if file_name.endswith('.txt'):  # Process any .txt file
                file_path = os.path.join(root, file_name)
               
                
                with open(file_path, 'r') as f:
                    current_sentence, current_labels = [], []
                    for line in f:
                        word_label = line.strip().split()
                        if len(word_label) == 2:
                            word, label = word_label
                            current_sentence.append(word)
                            current_labels.append(label)
                        elif len(current_sentence) > 0:
                            # Sentence boundary: Append sentence and its labels
                            sentences.append(' '.join(current_sentence))
                            labels.append(','.join(current_labels))
                           
                            current_sentence, current_labels = [], []
                    if len(current_sentence) > 0:
                        # Add the last sentence (if any) when file ends
                        sentences.append(' '.join(current_sentence))
                        labels.append(','.join(current_labels))
                   

    return pd.DataFrame({'sentence': sentences, 'word_labels': labels})


In [4]:
train_df= ingest_idrisi_data('Floods')

test_california= ingest_idrisi_data('Test/california_wildfires_2018_loc_notok')
test_idai= ingest_idrisi_data('Test/cyclone_idai_2019_loc_notok')
test_pakistan= ingest_idrisi_data('Test/pakistan_earthquake_2019_loc_notok')
test_dorian= ingest_idrisi_data('Test/hurricane_dorian_2019_loc_notok')
test_midwestern= ingest_idrisi_data('Test/midwestern_us_floods_2019_loc_notok')

In [5]:
def unify_loc_labels(df):
    # Modify the 'word_labels' column to replace B-LOC, I-LOC, L-LOC, and U-LOC with LOC
    df['word_labels'] = df['word_labels'].apply(lambda x: re.sub(r'\b[BILU]-LOC\b', 'LOC', x))
    return df

In [6]:
train_df= unify_loc_labels(train_df)

test_california= unify_loc_labels(test_california)
test_midwestern= unify_loc_labels(test_midwestern)
test_pakistan= unify_loc_labels(test_pakistan)
test_dorian= unify_loc_labels(test_dorian)
test_idai= unify_loc_labels(test_idai)

In [7]:
train_df

,sentence,word_labels
0,"DEVELOPING : Flash flooding , water rescues re...","O,O,O,O,O,O,O,O,O,LOC,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,"# EllicottCityFlood happened once again , kill...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,Searchers locate body of guardsman who went mi...,"O,O,O,O,O,O,O,O,O,LOC,O,O,O,O,O,O,O"
3,RT @TearsaSmith : Hero . Died trying to save a...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
4,Flags lowered to half - staff for Sgt . Eddiso...,"O,O,O,O,O,O,O,O,O,O,O,O,LOC,O,LOC,LOC,O,O,O,O,..."
...,...,...
1747,Kerala floods : Jalandhar Gurudwara to contrib...,"LOC,O,O,O,O,O,O,O,O,O,O,LOC,O,O,O,O,O,O,O,O,O,..."
1748,RT @CAChirag : Should Central Govt Accept Fina...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,LOC"
1749,How Kerala ’s prisoners were cooking food for ...,"O,LOC,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1750,Centre clarifies : Rs 600 crore released so fa...,"O,O,O,O,O,O,O,O,O,O,O,O,O,LOC,O,O,O,O,O,O,O,O,..."


# PRE-PROCESSING ????

In [13]:
# Check if any sentences have a mismatch in word and label count
train_df['sentence_len'] = train_df['sentence'].apply(lambda x: len(x.split()))
train_df['label_len'] = train_df['word_labels'].apply(lambda x: len(x.split(',')))

# Identify mismatches
mismatched = train_df[train_df['sentence_len'] != train_df['label_len']]
if not mismatched.empty:
    print("Mismatched entries found!")
    print(mismatched)




Before fine-tuning, it’s essential to map the location mention labels from the BILOU format to a format that BERT can understand. This involves converting categorical labels (e.g., B-CITY,B-CNTY, B-CONT) into integer IDs, which the model will use during training. This mapping is critical because BERT outputs logits for each token, which are then converted back to these labels.

In [14]:
# Extract unique tags from word labels
tags = set(",".join(train_df.word_labels).split(','))

# Create label to ID and ID to label mappings
label2id = {k: v for v, k in enumerate(tags)}
id2label = {v: k for v, k in enumerate(tags)}

In [15]:
label2id

{'LOC': 0, 'O': 1}

# BERT works on wordpiece tokenization rather than word tokenization. This means that we should also define the labels at the wordpiece-level, rather than the word-level!


In [16]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels


In [17]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]  
        word_labels = self.data.word_labels[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [18]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [19]:
training_set = dataset(train_df, tokenizer, MAX_LEN)

testset_california=  dataset(test_california, tokenizer, MAX_LEN)
testset_idai= dataset(test_idai, tokenizer, MAX_LEN)
testset_pakistan= dataset(test_pakistan, tokenizer, MAX_LEN)
testset_dorian= dataset(test_dorian, tokenizer, MAX_LEN)
testset_midwestern= dataset(test_midwestern, tokenizer, MAX_LEN)

In [20]:
# pytorch data loaders
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'num_workers': 0
                }

training_loader= DataLoader(training_set, **train_params)

testing_california= DataLoader(testset_california, **test_params)
testing_idai= DataLoader(testset_idai, **test_params)
testing_pakistan= DataLoader(testset_pakistan, **test_params)
testing_dorian= DataLoader(testset_dorian, **test_params)
testing_midwestern= DataLoader(testset_midwestern, **test_params)


In [21]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)


model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [22]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [28]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
   
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [29]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.706308901309967
Training loss per 100 training steps: 0.09073509408234812
Training loss per 100 training steps: 0.055791513796256326
Training loss per 100 training steps: 0.0438971798828311
Training loss per 100 training steps: 0.036756457851507565
Training loss epoch: 0.03545229639735745
Training accuracy epoch: 0.9726331162865973


In [30]:
# model.save_pretrained("ner_model_floods")
# tokenizer.save_pretrained("tokenizer_floods")



('tokenizer_floods/tokenizer_config.json',
 'tokenizer_floods/special_tokens_map.json',
 'tokenizer_floods/vocab.txt',
 'tokenizer_floods/added_tokens.json')

In [31]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Load the saved model
model = AutoModelForTokenClassification.from_pretrained("ner_model_floods")
model.to(device)

# Load the saved tokenizer
tokenizer = AutoTokenizer.from_pretrained("tokenizer_floods")


In [32]:
from transformers import pipeline
# THIS IS HAPPENING BECAUSE PIPELINE IS ONLY FOR IOB FORMATS RIGHT NOW!!!!!

nlp = pipeline("token-classification",model=model.to(device),tokenizer=tokenizer, aggregation_strategy="simple")
example = ("Floods detected in the Raipur district")
nlp(example)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity_group': 'LOC',
  'score': np.float32(0.56914014),
  'word': 'rai',
  'start': 23,
  'end': 26}]

In [33]:
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import numpy as np

def valid(model, testing_loader, tokenizer, id2label):
    # Put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps = 0
    all_results = []  # To store results for each tweet
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            ids = batch['ids'].to(model.device)
            mask = batch['mask'].to(model.device)
            targets = batch['targets'].to(model.device)
            
            # Forward pass through the model
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()
            nb_eval_steps += 1
            
            # Print intermediate loss every 100 steps
            if idx % 100 == 0:
                loss_step = eval_loss / nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # Compute predictions and mask out special tokens
            flattened_targets = targets.view(-1)
            active_logits = eval_logits.view(-1, model.num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)
            
            active_accuracy = mask.view(-1) == 1
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            # Convert token IDs back to tokens
            tokens = tokenizer.convert_ids_to_tokens(ids[0])  # For the first batch
            
            tweet_results = []
            for i, (token, label, pred) in enumerate(zip(tokens, targets, predictions)):
                label_str = id2label[label.item()]  # Get label string from label id
                pred_str = id2label[pred.item()]  # Get prediction string from predicted id
                
                # Get probabilities for each token's predicted label (apply softmax)
                token_probs = F.softmax(active_logits[i], dim=-1).cpu().numpy()  # Probabilities for the current token
                token_pred_prob = token_probs[np.argmax(token_probs)]  # Probability for the predicted label
                
                tweet_results.append({
                    'token': token,
                    'label': label_str,
                    'prediction': pred_str,
                    'proba': token_pred_prob  # Include the predicted probability for this token
                })
            
            # Add results for this tweet
            all_results.append(tweet_results)
            
            # Update accuracy
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    # Finalize average loss and accurac
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return all_results  # This will return a list of dictionaries for each tweet


In [34]:
#STEP 1: VALIDATION

# all_results_california= valid(model, testing_california, tokenizer,id2label)
# all_results_idai= valid(model, testing_idai, tokenizer,id2label)
# all_results_pakistan= valid(model, testing_pakistan, tokenizer,id2label)
# all_results_dorian= valid(model, testing_dorian, tokenizer,id2label)
# all_results_midwestern= valid(model, testing_midwestern, tokenizer,id2label)

Validation loss per 100 evaluation steps: 0.0030276982579380274
Validation loss per 100 evaluation steps: 0.014879734196589504
Validation loss per 100 evaluation steps: 0.014560579692954151
Validation loss per 100 evaluation steps: 0.0136558204139753
Validation loss per 100 evaluation steps: 0.013977698778978635
Validation loss per 100 evaluation steps: 0.01315283681279487
Validation loss per 100 evaluation steps: 0.013442409457735711
Validation loss per 100 evaluation steps: 0.012936167004528175
Validation loss per 100 evaluation steps: 0.01294258873935582
Validation loss per 100 evaluation steps: 0.01299699240601358
Validation loss per 100 evaluation steps: 0.01283958351988993
Validation loss per 100 evaluation steps: 0.013048182332128639
Validation Loss: 0.013006471563628118
Validation Accuracy: 0.9872194066443639
Validation loss per 100 evaluation steps: 0.016167152673006058
Validation loss per 100 evaluation steps: 0.020457647925261224
Validation loss per 100 evaluation steps: 0.0

In [35]:
import pickle
import numpy as np

def save_all_results(all_results, filename=None):
    """
    Saves the all_results list to a pickle file with a custom filename.
    
    Parameters:
        all_results (list): The list containing token, label, prediction, and proba for each token in each tweet.
        filename (str, optional): The filename to save the pickle file. Default is None, which saves to 'all_results.pkl'.
    """
    if filename is None:
        filename = "all_results.pkl"
    
    with open(filename, 'wb') as f:
        pickle.dump(all_results, f)
    print(f"All results saved to {filename}")

def load_all_results(filename=None):
    """
    Loads the all_results list from a pickle file with a custom filename.
    
    Parameters:
        filename (str, optional): The filename to load the pickle file from. Default is None, which loads from 'all_results.pkl'.
    
    Returns:
        list: A list containing token, label, prediction, and proba for each token in each tweet.
    """
    if filename is None:
        filename = "all_results.pkl"
    
    with open(filename, 'rb') as f:
        all_results = pickle.load(f)
    print(f"All results loaded from {filename}")
    return all_results


In [36]:
#Step 2: Saving these validation results
# save_all_results(all_results_california, filename="all_results_california_flood.pkl")
# save_all_results(all_results_idai, filename="all_results_idai_flood.pkl")
# save_all_results(all_results_pakistan, filename="all_results_pakistan_flood.pkl")
# save_all_results(all_results_dorian, filename="all_results_dorian_flood.pkl")
# save_all_results(all_results_midwestern, filename="all_results_midwestern_flood.pkl")

All results saved to all_results_california_flood.pkl
All results saved to all_results_idai_flood.pkl
All results saved to all_results_pakistan_flood.pkl
All results saved to all_results_dorian_flood.pkl
All results saved to all_results_midwestern_flood.pkl


In [37]:
#Step 3: Loading the validation result
all_results_california=load_all_results(filename="all_results_california_flood.pkl")
all_results_idai = load_all_results(filename="all_results_idai_flood.pkl")
all_results_pakistan = load_all_results(filename="all_results_pakistan_flood.pkl")
all_results_dorian = load_all_results(filename="all_results_dorian_flood.pkl")
all_results_midwestern = load_all_results(filename="all_results_midwestern_flood.pkl")

All results loaded from all_results_california_flood.pkl
All results loaded from all_results_idai_flood.pkl
All results loaded from all_results_pakistan_flood.pkl
All results loaded from all_results_dorian_flood.pkl
All results loaded from all_results_midwestern_flood.pkl


In [38]:
from sklearn.metrics import f1_score, classification_report

def calculate_f1_and_report(results):
    all_true_labels = []
    all_preds = []

    for tweet_result in results:
        for item in tweet_result:
            token = item['token']
            true_label = item['label']
            pred_label = item['prediction']

            # Exclude special tokens: [CLS], [SEP], and [PAD]
            if token not in ['[CLS]', '[SEP]', '[PAD]']:  # Exclude special tokens
                all_true_labels.append(true_label)
                all_preds.append(pred_label)
    
    # Calculate F1 score for the labels
    f1 = f1_score(all_true_labels, all_preds, average='micro')  # Use 'weighted' for multi-class tasks

    # Print the classification report
    report = classification_report(all_true_labels, all_preds)
    print("Classification Report:")
    print(report)

    return f1


In [39]:
#Step 4: F1 Score calculation
results=[all_results_california, all_results_idai, all_results_pakistan, all_results_dorian, all_results_midwestern]
for i in range(len(results)):
    print(calculate_f1_and_report(results[i]))

Classification Report:
              precision    recall  f1-score   support

         LOC       0.87      0.76      0.81      1510
           O       0.99      1.00      0.99     38639

    accuracy                           0.99     40149
   macro avg       0.93      0.88      0.90     40149
weighted avg       0.99      0.99      0.99     40149

0.9867493586390694
Classification Report:
              precision    recall  f1-score   support

         LOC       0.88      0.62      0.73      2210
           O       0.98      1.00      0.99     45463

    accuracy                           0.98     47673
   macro avg       0.93      0.81      0.86     47673
weighted avg       0.98      0.98      0.98     47673

0.9784574077570113
Classification Report:
              precision    recall  f1-score   support

         LOC       0.89      0.41      0.56      2123
           O       0.94      0.99      0.97     19658

    accuracy                           0.94     21781
   macro avg       0.